TO DO
- Develop code to handle no game / double game gameweeks (see, e.g. funtion get_was_home)

In [2]:
# number of latest gameweek
gameweek = 15
# check number of total players from fpl's site
total_players = 10876416

In [3]:
import pandas as pd
import numpy as np

import json
import requests
import sys

# allow more data columns to be shown than by default
pd.set_option('display.max_columns', 500)

In [4]:
sys.path.append('..')

In [5]:
import py_scripts.download_data as download_data

# Fetch FPL data

In [6]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

In [7]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
#fpl_data.set_index('id',inplace=True)
fpl_data['season'] = '2022-23'
fpl_data['name'] = fpl_data['first_name'] + ' ' + fpl_data['second_name']
# normalize spelling of names
fpl_data['name'] = fpl_data['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
#fpl_data['name'] = fpl_data['name'].map(player_name_dict).fillna(fpl_data['name'])
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
fpl_data['position'] = fpl_data['element_type'].map(position_dict)
# change team number to team name
fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]
fpl_data['GW'] = gameweek
fpl_data['value'] = fpl_data['now_cost']/10.0
fpl_data['selected'] = (fpl_data['selected_by_percent'].astype(float) / 100) * total_players
fpl_data['transfers_balance'] = fpl_data['transfers_in_event'] - fpl_data['transfers_out_event']
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance
0,100.0,100.0,58822,0,0,-3,3,0,2,0.7,0.2,0,Cédric,0.2,1,False,,2022-09-16T13:30:06.551108Z,42,58822.jpg,1.0,Alves Soares,0.1,False,None,a,Arsenal,3,1,5609,123,25427,253,0.0,0.2,Cédric,27,0,0,0,0,0,0,0,0,0,0,0,2,2.4,1.6,0.0,0.4,0,0.00000,0.00000,0.00000,0.00000,421,158,383,140,467,189,425,158,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.00,539,157,354,121,367,134,400,148,0.0,0.00000,2022-23,Cedric Alves Soares,DEF,15,4.2,10876.416,-130
1,NaN,NaN,84450,0,0,1,-1,1,3,5.3,4.8,3,Granit,4.8,3,False,,None,51,84450.jpg,4.7,Xhaka,3.9,False,None,a,Arsenal,3,61,665283,14695,418904,23520,0.9,12.0,Xhaka,1151,3,3,6,11,0,0,0,2,0,0,7,250,291.8,278.3,228.0,79.8,0,0.00000,0.00000,0.00000,0.00000,47,19,32,25,57,35,30,21,NaN,,3.0,,NaN,,0.0,0.0,0.0,0.0,0.0,0.86,170,99,38,14,33,16,82,27,0.0,0.46916,2022-23,Granit Xhaka,MID,15,5.1,424180.224,-8825
2,100.0,100.0,153256,0,0,-3,3,1,3,0.7,0.2,1,Mohamed,0.2,4,False,,2022-08-30T11:30:06.278675Z,42,153256.jpg,1.5,Elneny,0.7,False,None,a,Arsenal,3,3,138204,9486,198151,4700,0.0,0.7,Elneny,93,0,0,0,1,0,0,0,0,0,0,0,18,4.2,5.3,0.0,1.0,0,0.00000,0.00000,0.00000,0.00000,409,179,359,170,473,183,406,181,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.97,546,282,357,166,319,140,216,68,0.0,0.00000,2022-23,Mohamed Elneny,MID,15,4.2,76134.912,4786
3,NaN,NaN,156074,0,0,-3,3,0,2,1.0,0.5,1,Rob,0.5,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,Arsenal,3,4,5520,147,15645,244,0.1,1.0,Holding,12,0,0,0,0,0,0,0,0,0,0,0,14,2.4,0.1,0.0,0.2,0,0.00000,0.00000,0.00000,0.00000,423,160,422,158,474,194,435,161,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.00,547,163,312,103,370,137,420,156,0.0,0.00000,2022-23,Rob Holding,DEF,15,4.2,10876.416,-97
4,100.0,100.0,167199,0,0,-2,2,0,3,4.7,4.2,3,Thomas,4.2,6,False,,2022-09-25T09:00:06.484502Z,48,167199.jpg,4.0,Partey,0.5,False,None,a,Arsenal,3,40,74461,3226,92656,2559,0.9,8.3,Partey,869,2,0,6,6,0,0,0,0,0,0,4,182,208.6,128.1,111.0,44.8,0,0.00000,0.00000,0.00000,0.00000,101,42,116,83,144,91,112,70,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.62,261,159,57,24,62,29,242,77,0.0,0.62140,2022-23,Thomas Partey,MID,15,4.8,54382.080,667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,0.0,0.0,429414,0,0,-2,2,0,4,0.0,0.0,0,Sasa,0.0,608,False,Knee injury - Expected back 01 Jun,2022-09-03T18:00:06.993294Z,53,429414.jpg,1.0,Kalaj

In [8]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home_team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away_team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
fpl_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
0,2292871,NaN,False,False,61,None,0,False,None,8,NaN,1,NaN,[],2,4,74971,Arsenal,Everton
1,2292921,NaN,False,False,111,None,0,False,None,13,NaN,1,NaN,[],5,4,75021,Arsenal,Manchester City
2,2292870,NaN,False,False,62,None,0,False,None,5,NaN,3,NaN,[],3,2,74972,Bournemouth,Brighton
3,2292882,NaN,False,False,73,None,0,False,None,7,NaN,5,NaN,[],2,3,74983,Brighton,Crystal Palace
4,2292883,NaN,False,False,74,None,0,False,None,12,NaN,6,NaN,[],4,4,74984,Chelsea,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2293185,38.0,False,False,376,2023-05-28T15:00:00Z,0,False,False,3,NaN,8,NaN,[],2,2,75286,Everton,Bournemouth
376,2293186,38.0,False,False,377,2023-05-28T15:00:00Z,0,False,False,18,NaN,11,NaN,[],3,2,75287,Leeds United,Tottenham
377,2293187,38.0,False,False,378,2023-05-28T15:00:00Z,0,False,False,19,NaN,10,NaN,[],2,2,75288,Leicester City,West Ham
378,2293188,38.0,False,False,379,2023-05-28T15:00:00Z,0,False,False,9,NaN,14,NaN,[],2,4,75289,Manchester Utd,Fulham


In [9]:
def get_opponent(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    # my_team is the home team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team), \
                                         'away_team']
    # my_team is the away team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team), \
                                         'home_team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) )].shape[0]!=1:
        player = my_row['name']
        print(f'Number of games not equal to one for {player} ({my_team})')
        return np.nan

    return opponent_team.values[0]

In [10]:
def get_kickoff_time(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    kickoff_time = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) ), 'kickoff_time']
    if kickoff_time.shape[0]==1:
        return kickoff_time.values[0]
    else:
        player = my_row['name']
        print(f'Kickoff_time shape not one for {player} ({my_team})')
        return np.nan

In [11]:
def get_was_home(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        return 1
    elif fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        return 0
    else:
        player = my_row['name']
        print(f'Error in get_was_home for {player} ({my_team})')
        return np.nan

In [12]:
fpl_data['opp_team_name'] = fpl_data.apply(lambda x: get_opponent(x, fpl_fixtures, gameweek), axis=1)
fpl_data['kickoff_time'] = fpl_data.apply(lambda x: get_kickoff_time(x, fpl_fixtures, gameweek), axis=1)
fpl_data['was_home'] = fpl_data.apply(lambda x: get_was_home(x, fpl_fixtures, gameweek), axis=1)
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,100.0,100.0,58822,0,0,-3,3,0,2,0.7,0.2,0,Cédric,0.2,1,False,,2022-09-16T13:30:06.551108Z,42,58822.jpg,1.0,Alves Soares,0.1,False,None,a,Arsenal,3,1,5609,123,25427,253,0.0,0.2,Cédric,27,0,0,0,0,0,0,0,0,0,0,0,2,2.4,1.6,0.0,0.4,0,0.00000,0.00000,0.00000,0.00000,421,158,383,140,467,189,425,158,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.00,539,157,354,121,367,134,400,148,0.0,0.00000,2022-23,Cedric Alves Soares,DEF,15,4.2,10876.416,-130,Chelsea,2022-11-06T12:00:00Z,0
1,NaN,NaN,84450,0,0,1,-1,1,3,5.3,4.8,3,Granit,4.8,3,False,,None,51,84450.jpg,4.7,Xhaka,3.9,False,None,a,Arsenal,3,61,665283,14695,418904,23520,0.9,12.0,Xhaka,1151,3,3,6,11,0,0,0,2,0,0,7,250,291.8,278.3,228.0,79.8,0,0.00000,0.00000,0.00000,0.00000,47,19,32,25,57,35,30,21,NaN,,3.0,,NaN,,0.0,0.0,0.0,0.0,0.0,0.86,170,99,38,14,33,16,82,27,0.0,0.46916,2022-23,Granit Xhaka,MID,15,5.1,424180.224,-8825,Chelsea,2022-11-06T12:00:00Z,0
2,100.0,100.0,153256,0,0,-3,3,1,3,0.7,0.2,1,Mohamed,0.2,4,False,,2022-08-30T11:30:06.278675Z,42,153256.jpg,1.5,Elneny,0.7,False,None,a,Arsenal,3,3,138204,9486,198151,4700,0.0,0.7,Elneny,93,0,0,0,1,0,0,0,0,0,0,0,18,4.2,5.3,0.0,1.0,0,0.00000,0.00000,0.00000,0.00000,409,179,359,170,473,183,406,181,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.97,546,282,357,166,319,140,216,68,0.0,0.00000,2022-23,Mohamed Elneny,MID,15,4.2,76134.912,4786,Chelsea,2022-11-06T12:00:00Z,0
3,NaN,NaN,156074,0,0,-3,3,0,2,1.0,0.5,1,Rob,0.5,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,Arsenal,3,4,5520,147,15645,244,0.1,1.0,Holding,12,0,0,0,0,0,0,0,0,0,0,0,14,2.4,0.1,0.0,0.2,0,0.00000,0.00000,0.00000,0.00000,423,160,422,158,474,194,435,161,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.00,547,163,312,103,370,137,420,156,0.0,0.00000,2022-23,Rob Holding,DEF,15,4.2,10876.416,-97,Chelsea,2022-11-06T12:00:00Z,0
4,100.0,100.0,167199,0,0,-2,2,0,3,4.7,4.2,3,Thomas,4.2,6,False,,2022-09-25T09:00:06.484502Z,48,167199.jpg,4.0,Partey,0.5,False,None,a,Arsenal,3,40,74461,3226,92656,2559,0.9,8.3,Partey,869,2,0,6,6,0,0,0,0,0,0,4,182,208.6,128.1,111.0,44.8,0,0.00000,0.00000,0.00000,0.00000,101,42,116,83,144,91,112,70,NaN,,NaN,,NaN,,0.0,0.0,0.0,0.0,0.0,0.62,261,159,57,24,62,29,242,77,0.0,0.62140,2022-23,Thomas Partey,MID,15,4.8,54382.080,667,Chelsea,2022-11-06T12:00:00Z,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [13]:
fpl_data.to_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv')
fpl_fixtures.to_csv('../data/fpl_fixtures.csv')

In [14]:
difference_columns = [
    'total_points', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
    'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed',
    'penalties_saved', 'saves', 'threat', 'yellow_cards'
]

In [15]:
def calculate_difference_to_previous_gameweek(my_row):
    my_id = my_row['id']
    if my_id in old_ids:
        for my_column in difference_columns:
            my_row[my_column] = (my_row[my_column] -
                                     fpl_previous.loc[fpl_previous['id']==my_id, my_column]).values[0]
        return my_row
    elif my_id in new_ids:
        # for new players, these should be the same already, but just in case
        my_row['total_points'] = my_row['event_points']
        return my_row
    else:
        print(f'Player id neither old or new: {my_id}')
        return np.nan

In [16]:
# create a csv that contains each week's fpl data in one table
if gameweek==1:
    fpl_data['total_points'] = fpl_data['event_points']
    fpl_data[fpl_data['minutes']>0].to_csv('../data/fpl_data.csv')
    print(f'Gameweek {gameweek} data saved.')
elif gameweek>1:
    fpl_previous = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek-1}.csv')
    new_ids = set(fpl_data['id']).difference(set(fpl_previous['id']))
    old_ids = set(fpl_data['id']).intersection(set(fpl_previous['id']))
    # convert all difference columns to floats
    fpl_data[difference_columns] = fpl_data[difference_columns].astype(float)
    fpl_gameweek = fpl_data.apply(lambda x: calculate_difference_to_previous_gameweek(x), axis=1)
    fpl_gameweek = fpl_gameweek[fpl_gameweek['minutes']>0]
    print(f'Number of players that played this week: {fpl_gameweek.shape[0]}')
    fpl_previous_weeks = pd.read_csv('../data/fpl_data.csv', index_col=0)
    print(f'Size of fpl_previous_weeks: {fpl_previous_weeks.shape[0]}')
    #fpl_previous_weeks = fpl_previous_weeks[fpl_previous_weeks['GW']<gameweek]
    fpl_all_weeks = pd.concat([fpl_previous_weeks, fpl_gameweek], axis=0)
    print(f'Size of fpl_all_weeks: {fpl_all_weeks.shape[0]}')
    display(fpl_all_weeks)
else:
    print('Check gameweek!')

Number of players that played this week: 297
Size of fpl_previous_weeks: 3739
Size of fpl_all_weeks: 4036


,id,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,starts_per_90,clean_sheets_per_90
0,3,NaN,NaN,84450,0,0,0,0,0,3,2.1,2.1,2,Granit,2.0,False,NaN,NaN,50,84450.jpg,2.0,Xhaka,0.7,False,NaN,a,Arsenal,3,2.0,8050,8050,8788,8788,0.4,0.4,Xhaka,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,12.0,16.6,15.0,2.0,3.4,98,32,81,52,167,90,123,65,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Granit Xhaka,MID,1,5.0,60582.963,-738,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,167199,0,0,0,0,0,3,2.1,2.1,3,Thomas,3.0,False,NaN,NaN,50,167199.jpg,3.0,Partey,0.7,False,NaN,a,Arsenal,3,3.0,11320,11320,7903,7903,0.6,0.6,Partey,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,17.0,22.6,12.9,10.0,4.6,61,23,95,58,96,57,85,48,NaN,NaN,4.0,NaN,NaN,NaN,2022-23,Thomas Partey,MID,1,5.0,60582.963,3417,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,184029,0,0,0,0,0,3,2.8,2.8,3,Martin,3.0,False,NaN,NaN,65,184029.jpg,3.0,Ødegaard,13.4,False,NaN,a,Arsenal,3,3.0,52695,52695,121593,121593,0.5,0.5,Ødegaard,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,9.0,3.4,8.3,23.0,3.5,220,96,116,68,52,29,119,64,3.0,NaN,1.0,NaN,NaN,NaN,2022-23,Martin degaard,MID,1,6.5,1159731.006,-68898,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,100.0,75.0,192895,0,0,0,0,0,2,2.8,2.1,1,Kieran,1.0,False,NaN,2022-08-04T15:00:05.276574Z,50,192895.jpg,1.0,Tierney,1.2,False,NaN,a,Arsenal,3,1.0,2682,2682,16585,16585,0.2,0.2,Tierney,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3.0,6.0,0.2,0.0,0.6,195,85,249,88,491,201,245,88,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Kieran Tierney,DEF,1,5.0,103856.508,-13903,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,NaN,NaN,198869,0,0,0,0,0,2,2.4,2.4,5,Benjamin,5.0,False,NaN,NaN,45,198869.jpg,5.0,White,6.4,False,NaN,a,Arsenal,3,5.0,30058,30058,35607,35607,1.1,1.1,White,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,25.0,31.4,12.0,0.0,4.3,36,10,99,32,349,150,90,24,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Ben White,DEF,1,4.5,553901.376,-5549,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [17]:
# save data
if gameweek>1:
    fpl_all_weeks.to_csv('../data/fpl_data.csv')

# Fetch fbref data

In [18]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
links = download_data.get_match_report_links(url)
links

['https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 'https://fbref.com/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/1ac96eb4/Newcastle-United-Nottingham-Forest-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/82702941/Leeds-United-Wolverhampton-Wanderers-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/877e3193/Bournemouth-Aston-Villa-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3a917cee/Everton-Chelsea-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3249ba27/Leicester-City-Brentford-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/8251694e/Manchester-United-Brighton-and-Hove-Albion-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/ece62baf/West-Ham-United-Manchester-City-August-7-2022-Premier-League',
 'https:

In [19]:
# check which matches (links) are already collected
with open('../data/fbref_fixture_links.txt') as f:
    old_links = f.read().splitlines()
old_links

['https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 'https://fbref.com/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/1ac96eb4/Newcastle-United-Nottingham-Forest-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/82702941/Leeds-United-Wolverhampton-Wanderers-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/877e3193/Bournemouth-Aston-Villa-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3a917cee/Everton-Chelsea-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3249ba27/Leicester-City-Brentford-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/8251694e/Manchester-United-Brighton-and-Hove-Albion-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/ece62baf/West-Ham-United-Manchester-City-August-7-2022-Premier-League',
 'https:

In [20]:
# get links to games for which data has not been collected yet
new_links = [link for link in links if link not in old_links]
new_links

['https://fbref.com/en/matches/12251835/Nottingham-Forest-Brentford-November-5-2022-Premier-League',
 'https://fbref.com/en/matches/63975c3e/Leeds-United-Bournemouth-November-5-2022-Premier-League',
 'https://fbref.com/en/matches/6ef27b3c/Manchester-City-Fulham-November-5-2022-Premier-League',
 'https://fbref.com/en/matches/982d16a2/Wolverhampton-Wanderers-Brighton-and-Hove-Albion-November-5-2022-Premier-League',
 'https://fbref.com/en/matches/831c73fc/Everton-Leicester-City-November-5-2022-Premier-League',
 'https://fbref.com/en/matches/af9dc838/North-West-London-Derby-Chelsea-Arsenal-November-6-2022-Premier-League',
 'https://fbref.com/en/matches/2586fb04/West-Ham-United-Crystal-Palace-November-6-2022-Premier-League',
 'https://fbref.com/en/matches/57f9bcc9/Southampton-Newcastle-United-November-6-2022-Premier-League',
 'https://fbref.com/en/matches/6638574f/Aston-Villa-Manchester-United-November-6-2022-Premier-League',
 'https://fbref.com/en/matches/2954504d/Tottenham-Hotspur-Liverpo

In [21]:
fbref_data = download_data.download_match_report_data(new_links)
# normalize spelling of player names
fbref_data[('General','Player')] = fbref_data[('General','Player')].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
# map certain player names such that they are the same for both fpl and fbref data
#fbref_data[('General','Player')] = fbref_data[('General','Player')].map(player_name_dict).fillna(fbref_data[('General','Player')])
fbref_data

100%|███████████████████████████████████████████| 10/10 [00:38<00:00,  3.88s/it]


General                                   Performance      \
                     Player     #   Nation    Pos     Age Min         Gls Ast   
0           Emmanuel Dennis  25.0   ng NGA     FW  24-355  78           0   1   
1             Taiwo Awoniyi   9.0   ng NGA     FW  25-085  12           0   0   
2           Brennan Johnson  20.0  wls WAL  FW,RW  21-166  90           0   0   
3        Morgan Gibbs-White  10.0  eng ENG  AM,RB  22-282  90           1   0   
4          Cheikhou Kouyate  21.0   sn SEN     LM  32-319  65           0   0   
..                      ...   ...      ...    ...     ...  ..         ...  ..   
292        Andrew Robertson  26.0  sct SCO     LB  28-240  90           0   0   
293         Virgil van Dijk   4.0   nl NED     CB  31-121  90           0   0   
294         Ibrahima Konate   5.0   fr FRA     CB  23-165  90           0   0   
295  Trent Alexander-Arnold  66.0  eng ENG     RB  24-030  90           0   0   
296                 Alisson   1.0   br BRA     GK  30-035  90           0   0   

                                                     Expected           SCA  \
    PK PKatt Sh SoT CrdY CrdR Touches Tkl Int Blocks       xG npxG  xAG SCA   
0    0     0  1   1    0    0      45   2   0      2      0.2  0.2  0.1   3   
1    0     0  1   0    0    0       3   0   0      0      0.2  0.2  0.0   0   
2    0     0  2   2    0    0      28   1   0      1      0.4  0.4  0.1   1   
3    0     0  7   2    0    0      54   1   0      0      0.5  0.5  0.4   5   
4    0     0  0   0    0    0      22   0   0      0      0.0  0.0  0.2   1   
..  ..   ... ..  ..  ...  ...     ...  ..  ..    ...      ...  ...  ...  ..   
292  0     0  0   0    0    0      85   1   2      0      0.0  0.0  0.1   7   
293  0     0  0   0    0    0      79   0   0      0      0.0  0.0  0.0   1   
294  0     0  0   0    0    0      90   3   2      0      0.0  0.0  0.0   1   
295  0     0  0   0    0    0      88   0   1      1      0.0  0.0  0.1   2   
296  0     0  0   0    0    0      49   0   0      0      0.0  0.0  0.0   0   

        Passes                Dribbles              General           \
    GCA    Cmp Att  Cmp% Prog     Succ Att            Squad HomeTeam   
0     1     20  27  74.1    3        1   4  Nott'ham Forest     True   
1     0      0   1   0.0    0        0   0  Nott'ham Forest     True   
2     0     13  22  59.1    0        0   1  Nott'ham Forest     True   
3     0     28  38  73.7    4        1   2  Nott'ham Forest     True   
4     0     15  18  83.3    1        3   3  Nott'ham Forest     True   
..   ..    ...  ..   ...  ...      ...  ..              ...      ...   
292   2     64  79  81.0    5        1   2        Liverpool    False   
293   1     66  72  91.7    1        0   0        Liverpool    False   
294   0     57  74  77.0    2        0   0        Liverpool    False   
295   0     60  78  76.9    4        0   0        Liverpool    False   
296   0     29  45  64.4    0        0   0        Liverpool    False   

               Total                           Short            Medium      \
      Opponent   Cmp Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%    Cmp Att   
0    Brentford    20  27  74.1     318      69    12  14   85.7      4   7   
1    Brentford     0   1   0.0       0       0     0   0    NaN      0   1   
2    Brentford    13  22  59.1     138      11    11  13   84.6      2   2   
3    Brentford    28  38  73.7     427     154    14  15   93.3     10  12   
4    Brentford    15  18  83.3     214      51     9  11   81.8      5   6   
..         ...   ...  ..   ...     ...     ...   ...  ..    ...    ...  ..   
292  Tottenham    64  79  81.0     951     330    38  41   92.7     23  27   
293  Tottenham    66  72  91.7    1215     404    23  26   88.5     37  38   
294  Tottenham    57  74  77.0    1108     373    17  23   73.9     37  43   
295  Tottenham    60  78  76.9    1012     530    37  43   86.0     16  23   
296  Tottenham    29  45  64.4     832     557     4   4  100.0  

In [22]:
# change column name to align the new Opta data with the old Statsbomb data
fbref_data.columns = fbref_data.columns.values
fbref_data.columns = pd.MultiIndex.from_tuples(fbref_data.rename(columns={('Expected', 'xAG'): ('Expected', 'xA')}))
fbref_data

General                                   Performance      \
                     Player     #   Nation    Pos     Age Min         Gls Ast   
0           Emmanuel Dennis  25.0   ng NGA     FW  24-355  78           0   1   
1             Taiwo Awoniyi   9.0   ng NGA     FW  25-085  12           0   0   
2           Brennan Johnson  20.0  wls WAL  FW,RW  21-166  90           0   0   
3        Morgan Gibbs-White  10.0  eng ENG  AM,RB  22-282  90           1   0   
4          Cheikhou Kouyate  21.0   sn SEN     LM  32-319  65           0   0   
..                      ...   ...      ...    ...     ...  ..         ...  ..   
292        Andrew Robertson  26.0  sct SCO     LB  28-240  90           0   0   
293         Virgil van Dijk   4.0   nl NED     CB  31-121  90           0   0   
294         Ibrahima Konate   5.0   fr FRA     CB  23-165  90           0   0   
295  Trent Alexander-Arnold  66.0  eng ENG     RB  24-030  90           0   0   
296                 Alisson   1.0   br BRA     GK  30-035  90           0   0   

                                                     Expected           SCA  \
    PK PKatt Sh SoT CrdY CrdR Touches Tkl Int Blocks       xG npxG   xA SCA   
0    0     0  1   1    0    0      45   2   0      2      0.2  0.2  0.1   3   
1    0     0  1   0    0    0       3   0   0      0      0.2  0.2  0.0   0   
2    0     0  2   2    0    0      28   1   0      1      0.4  0.4  0.1   1   
3    0     0  7   2    0    0      54   1   0      0      0.5  0.5  0.4   5   
4    0     0  0   0    0    0      22   0   0      0      0.0  0.0  0.2   1   
..  ..   ... ..  ..  ...  ...     ...  ..  ..    ...      ...  ...  ...  ..   
292  0     0  0   0    0    0      85   1   2      0      0.0  0.0  0.1   7   
293  0     0  0   0    0    0      79   0   0      0      0.0  0.0  0.0   1   
294  0     0  0   0    0    0      90   3   2      0      0.0  0.0  0.0   1   
295  0     0  0   0    0    0      88   0   1      1      0.0  0.0  0.1   2   
296  0     0  0   0    0    0      49   0   0      0      0.0  0.0  0.0   0   

        Passes                Dribbles              General           \
    GCA    Cmp Att  Cmp% Prog     Succ Att            Squad HomeTeam   
0     1     20  27  74.1    3        1   4  Nott'ham Forest     True   
1     0      0   1   0.0    0        0   0  Nott'ham Forest     True   
2     0     13  22  59.1    0        0   1  Nott'ham Forest     True   
3     0     28  38  73.7    4        1   2  Nott'ham Forest     True   
4     0     15  18  83.3    1        3   3  Nott'ham Forest     True   
..   ..    ...  ..   ...  ...      ...  ..              ...      ...   
292   2     64  79  81.0    5        1   2        Liverpool    False   
293   1     66  72  91.7    1        0   0        Liverpool    False   
294   0     57  74  77.0    2        0   0        Liverpool    False   
295   0     60  78  76.9    4        0   0        Liverpool    False   
296   0     29  45  64.4    0        0   0        Liverpool    False   

               Total                           Short            Medium      \
      Opponent   Cmp Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%    Cmp Att   
0    Brentford    20  27  74.1     318      69    12  14   85.7      4   7   
1    Brentford     0   1   0.0       0       0     0   0    NaN      0   1   
2    Brentford    13  22  59.1     138      11    11  13   84.6      2   2   
3    Brentford    28  38  73.7     427     154    14  15   93.3     10  12   
4    Brentford    15  18  83.3     214      51     9  11   81.8      5   6   
..         ...   ...  ..   ...     ...     ...   ...  ..    ...    ...  ..   
292  Tottenham    64  79  81.0     951     330    38  41   92.7     23  27   
293  Tottenham    66  72  91.7    1215     404    23  26   88.5     37  38   
294  Tottenham    57  74  77.0    1108     373    17  23   73.9     37  43   
295  Tottenham    60  78  76.9    1012     530    37  43   86.0     16  23   
296  Tottenham    29  45  64.4     832     557     4   4  100.0  

In [23]:
# save links that have been downloaded already so that in the future one can only fetch data from new games
with open('../data/fbref_fixture_links.txt', 'w') as f:
    f.write('\n'.join(links))

In [24]:
old_fbref_data = pd.read_csv('../data/fbref_data.csv', index_col=0, header=[0,1])
#old_fbref_data.to_csv(f'../data/fbref_data_backup_week{gameweek}.csv')

In [25]:
fbref_data = pd.concat([old_fbref_data, fbref_data], axis=0)
print(f'Number of rows in fbref_data: {fbref_data.shape[0]}.')

Number of rows in fbref_data: 4036.


In [26]:
fbref_data.to_csv(f'../data/fbref_data.csv')

## 22-23 Match results

In [27]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
data = pd.read_html(url)
results = data[0].dropna(axis=0, how='all').copy()
results

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,Match Report,NaN
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,Match Report,NaN
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,Match Report,NaN
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St. James' Park,Simon Hooper,Match Report,NaN
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,Match Report,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,38.0,Sun,2023-05-28,15:00,Everton,NaN,NaN,NaN,Bournemouth,NaN,Goodison Park,NaN,Head-to-Head,NaN
413,38.0,Sun,2023-05-28,15:00,Leicester City,NaN,NaN,NaN,West Ham,NaN,King Power Stadium,NaN,Head-to-Head,NaN
414,38.0,Sun,2023-05-28,15:00,Aston Villa,NaN,NaN,NaN,Brighton,NaN,Villa Park,NaN,Head-to-Head,NaN
415,38.0,Sun,2023-05-28,15:00,Leeds United,NaN,NaN,NaN,Tottenham,NaN,Elland Road,NaN,Head-to-Head,NaN


In [28]:
# fix Nott Forest team name
my_dict = {"Nott'ham Forest":'Nottingham Forest'}
results['Home'] = results['Home'].map(my_dict).fillna(results['Home'])
results['Away'] = results['Away'].map(my_dict).fillna(results['Away'])

In [29]:
results.to_csv('../data/PL_results_season22_23.csv')